In [2]:
from func.shopee_crawler import search_get_items, get_items_from_industry, get_in4_item

In [3]:
get_in4_item(439702972, 11402808589)

,shopid,itemid,brand,price,stock,discount,historical_sold,sold,description,item_rating.rating_star,images,categories,shop_vouchers,global_sold,lv1,lv2,lv3
0,439702972,11402808589,BeGreen House,16423122522,3268,50%,634,46,THÔNG TIN SẢN PHẨM SỮA TẮM CHO NAM BEGREEN HOU...,3.055968,"[01d1479a4b782e01054948aa97989a4b, 28ce82d7648...","[{'catid': 100630, 'display_name': 'Sắc Đẹp', ...",[],634,Sắc Đẹp,Chăm sóc nam giới,Sữa tắm & chăm sóc cơ thể
